<a href="https://colab.research.google.com/github/AbdellaouiHajar1/test2/blob/master/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
! pip install kaggle

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
! mkdir  ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [41]:
! cp /content/drive/MyDrive/ColabNotebooks/kaggle.json ~/.kaggle/kaggle.json

In [42]:
!chmod 600 ~/.kaggle/kaggle.json

In [43]:
! kaggle datasets download -d  nltkdata/movie-review

movie-review.zip: Skipping, found more recently modified local copy (use --force to force download)


In [44]:
! unzip /content/movie-review.zip

Archive:  /content/movie-review.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: README                  
  inflating: movie_review.csv        
  inflating: movie_reviews-meta.json  
  inflating: movie_reviews/movie_reviews/README  
  inflating: movie_reviews/movie_reviews/neg/cv000_29416.txt  
  inflating: movie_reviews/movie_reviews/neg/cv001_19502.txt  
  inflating: movie_reviews/movie_reviews/neg/cv002_17424.txt  
  inflating: movie_reviews/movie_reviews/neg/cv003_12683.txt  
  inflating: movie_reviews/movie_reviews/neg/cv004_12641.txt  
  inflating: movie_reviews/movie_reviews/neg/cv005_29357.txt  
  inflating: movie_reviews/movie_reviews/neg/cv006_17022.txt  
  inflating: movie_reviews/movie_reviews/neg/cv007_4992.txt  
  inflating: movie_reviews/movie_reviews/neg/cv008_29326.txt  
  inflating: movie_reviews/movie_reviews/neg/cv009_29417.txt  
  inflating: movie_reviews/movie_reviews/neg/cv010_29063.txt  
  inflating: movie_reviews/movie_reviews/neg/cv011_

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import sklearn
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [46]:
full_df=pd.read_csv('/content/movie_review.csv')
df = full_df[["text","tag"]]
df["text"] = df["text"].astype(str)
full_df.head()

<ipython-input-46-42a432a314bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].astype(str)


,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


***Pre-processing des données textuelles :***

In [47]:
#Mise en minuscules
df["text_lower"] = df["text"].str.lower()
df.head()

<ipython-input-47-5bf5a7484829>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_lower"] = df["text"].str.lower()


,text,tag,text_lower
0,films adapted from comic books have had plenty...,pos,films adapted from comic books have had plenty...
1,"for starters , it was created by alan moore ( ...",pos,"for starters , it was created by alan moore ( ..."
2,to say moore and campbell thoroughly researche...,pos,to say moore and campbell thoroughly researche...
3,"the book ( or "" graphic novel , "" if you will ...",pos,"the book ( or "" graphic novel , "" if you will ..."
4,"in other words , don't dismiss this film becau...",pos,"in other words , don't dismiss this film becau..."


In [48]:
#Suppression des ponctuations
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))
df.head()

<ipython-input-48-619de397f509>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))


,text,tag,text_lower,text_wo_punct
0,films adapted from comic books have had plenty...,pos,films adapted from comic books have had plenty...,films adapted from comic books have had plenty...
1,"for starters , it was created by alan moore ( ...",pos,"for starters , it was created by alan moore ( ...",for starters it was created by alan moore an...
2,to say moore and campbell thoroughly researche...,pos,to say moore and campbell thoroughly researche...,to say moore and campbell thoroughly researche...
3,"the book ( or "" graphic novel , "" if you will ...",pos,"the book ( or "" graphic novel , "" if you will ...",the book or graphic novel if you will is ...
4,"in other words , don't dismiss this film becau...",pos,"in other words , don't dismiss this film becau...",in other words dont dismiss this film because...


In [49]:
#stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-49-635a4a2e271e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))


,text,tag,text_lower,text_wo_punct,text_wo_stop
0,films adapted from comic books have had plenty...,pos,films adapted from comic books have had plenty...,films adapted from comic books have had plenty...,films adapted comic books plenty success wheth...
1,"for starters , it was created by alan moore ( ...",pos,"for starters , it was created by alan moore ( ...",for starters it was created by alan moore an...,starters created alan moore eddie campbell bro...
2,to say moore and campbell thoroughly researche...,pos,to say moore and campbell thoroughly researche...,to say moore and campbell thoroughly researche...,say moore campbell thoroughly researched subje...
3,"the book ( or "" graphic novel , "" if you will ...",pos,"the book ( or "" graphic novel , "" if you will ...",the book or graphic novel if you will is ...,book graphic novel 500 pages long includes nea...
4,"in other words , don't dismiss this film becau...",pos,"in other words , don't dismiss this film becau...",in other words dont dismiss this film because...,words dont dismiss film source


***Entraînement du modèle Word2Vec :***

In [50]:
preprocessed_reviews = [text for text in df["text_wo_stop"]]

# Entraînement du modèle Word2Vec
word2vec_model = Word2Vec([preprocessed_reviews], vector_size=100, window=5, min_count=1,sg=1)


***Vectorisation des reviews de movies :***

In [51]:
def vectorisation_review(review, word2vec_model):
    vector = np.mean([word2vec_model.wv[word] for word in review.split() if word in word2vec_model.wv], axis=0)
    return vector

# Appliquer la fonction à la colonne "text_lemmatized" du DataFrame
df["vectors"] = df["text_wo_stop"].apply(lambda review: vectorisation_review(review, word2vec_model))
df["vectors"]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


0        [-0.003004536, 0.004758916, -0.00092252356, -0...
1                                                      NaN
2        [-0.0045479755, 0.0038913414, -0.0006084019, -...
3        [-0.005362962, -0.0012496419, -0.004538166, 0....
4        [-0.0019622324, -0.0026777082, 0.0020702675, -...
                               ...                        
64715    [0.0060458053, 0.0010254853, -0.007881565, 0.0...
64716    [-0.00033637765, 0.0052027223, -0.003555229, -...
64717    [-0.00077041634, 0.00073529663, -0.005595822, ...
64718    [0.002273716, 0.002579851, -0.0011500336, -0.0...
64719                                                  NaN
Name: vectors, Length: 64720, dtype: object

***Division des données :***

In [52]:
df["tag"] = df["tag"].replace({"pos": 1, "neg": 0})
df.dropna(inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df["vectors"], df["tag"] , test_size=0.2)
X_train

38209    [0.0010900256, 0.0022966724, 0.004545473, 0.00...
14953    [0.0015024127, 0.007863428, -0.0071024196, 0.0...
43698    [0.0026081356, -9.090215e-05, -0.0010495633, 0...
46345    [-0.004109801, -0.004199336, -0.0040825447, 0....
63847    [-0.0033566868, -0.0018127026, -0.0025010568, ...
                               ...                        
55918    [-0.0066642477, -0.0074477177, 0.0034805269, -...
27965    [0.005626712, 0.005497371, 0.0018291199, 0.005...
62583    [-0.0026969297, 0.00068489317, 0.0022751251, -...
13074    [0.0043161227, -0.0020099923, -0.0018987567, -...
61143    [0.0006015388, 0.00083620695, -0.004679726, -0...
Name: vectors, Length: 46866, dtype: object

In [53]:
X_test

9746     [0.0012704676, 0.0025834478, 0.0021739148, -0....
20270    [-0.0007158604, 0.0024445588, -0.004217505, -0...
39614    [0.0053300434, -0.0042784493, -0.0041270917, -...
51670    [-0.0019435257, -0.00039861814, -0.0017104102,...
56218    [0.00041849155, 0.00038531236, 0.0016818652, -...
                               ...                        
49174    [0.00037056697, -0.00032213237, -0.002050832, ...
25950    [0.0019464601, -0.006360857, 0.008766697, -0.0...
29659    [0.0058129174, -0.0025993485, -0.00028986507, ...
930      [-0.002855879, -0.0011685812, 0.0084352065, 0....
22732    [0.0011628044, -0.0040671933, -0.002937547, 0....
Name: vectors, Length: 11717, dtype: object

***Construction d'un classificateur :***

In [54]:



classifier = LogisticRegression()
classifier.fit(list(X_train), y_train)

LogisticRegression()

***Évaluation du modèle :***

In [55]:
# Prédiction sur l'ensemble de test
predictions = classifier.predict(list(X_test))

# Évaluation du modèle
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.5410087906460698
Precision: 0.5333257247203834
Recall: 0.7837973834283798
F1-score: 0.634745992936702
